In [27]:
from data import *
from data import VOC_CLASSES as labelmap
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import os.path as osp
import sys
import time

import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import functional as F
import torchvision as tv
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets

import nntools as nt
import numpy as np
from PIL import Image
import argparse
from matplotlib import pyplot as plt
import pandas as pd
import cv2 
import pickle
import random
import tarfile
import collections

In [5]:
if sys.version_info[0] == 2:
    import xml.etree.cElementTree as ET
else:
    import xml.etree.ElementTree as ET

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [7]:
class Timer(object):
    """A simple timer."""
    def __init__(self):
        self.total_time = 0.
        self.calls = 0
        self.start_time = 0.
        self.diff = 0.
        self.average_time = 0.

    def tic(self):
        # using time.time instead of time.clock because time time.clock
        # does not normalize for multithreading
        self.start_time = time.time()

    def toc(self, average=True):
        self.diff = time.time() - self.start_time
        self.total_time += self.diff
        self.calls += 1
        self.average_time = self.total_time / self.calls
        if average:
            return self.average_time
        else:
            return self.diff

In [8]:
def parse_rec(filename):
    """ Parse a PASCAL VOC xml file """
    tree = ET.parse(filename)
    objects = []
    for obj in tree.findall('object'):
        obj_struct = {}
        obj_struct['name'] = obj.find('name').text
        obj_struct['pose'] = obj.find('pose').text
        obj_struct['truncated'] = int(obj.find('truncated').text)
        obj_struct['difficult'] = int(obj.find('difficult').text)
        bbox = obj.find('bndbox')
        obj_struct['bbox'] = [int(bbox.find('xmin').text) - 1,
                              int(bbox.find('ymin').text) - 1,
                              int(bbox.find('xmax').text) - 1,
                              int(bbox.find('ymax').text) - 1]
        objects.append(obj_struct)

    return objects

In [9]:
def get_voc_results_file_template(image_set, cls):
    # VOCdevkit/VOC2007/results/det_test_aeroplane.txt
    filename = 'det_' + image_set + '_%s.txt' % (cls)
    filedir = os.path.join(devkit_path, 'results')
    if not os.path.exists(filedir):
        os.makedirs(filedir)
    path = os.path.join(filedir, filename)
    return path


def write_voc_results_file(all_boxes, dataset):
    for cls_ind, cls in enumerate(labelmap):
        print('Writing {:s} VOC results file'.format(cls))
        filename = get_voc_results_file_template(set_type, cls)
        
        with open(filename, 'wt') as f:
            for im_ind, index in enumerate(dataset.ids):
                dets = all_boxes[cls_ind+1][im_ind]
                if dets == []:
                    continue
                # the VOCdevkit expects 1-based indices
                for k in range(dets.shape[0]):
                    f.write('{:s} {:.3f} {:.1f} {:.1f} {:.1f} {:.1f}\n'.
                            format(index[1], dets[k, -1],
                                   dets[k, 0] + 1, dets[k, 1] + 1,
                                   dets[k, 2] + 1, dets[k, 3] + 1))


def do_python_eval(output_dir='output', use_07=True):
    cachedir = os.path.join(devkit_path, 'annotations_cache')
    aps = []
    
    # The PASCAL VOC metric changed in 2010
    use_07_metric = use_07
    print('VOC07 metric? ' + ('Yes' if use_07_metric else 'No'))
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
        
    for i, cls in enumerate(labelmap):
        filename = get_voc_results_file_template(set_type, cls)
        rec, prec, ap = voc_eval(
           filename, annopath, imgsetpath.format(set_type), cls, cachedir,
           ovthresh=0.5, use_07_metric=use_07_metric)
        aps += [ap]
        print('AP for {} = {:.4f}'.format(cls, ap))
        with open(os.path.join(output_dir, cls + '_pr.pkl'), 'wb') as f:
            pickle.dump({'rec': rec, 'prec': prec, 'ap': ap}, f)
    print('Mean AP = {:.4f}'.format(np.mean(aps)))
    print('~~~~~~~~')
    print('Results:')
    for ap in aps:
        print('{:.3f}'.format(ap))
    print('{:.3f}'.format(np.mean(aps)))
    print('~~~~~~~~')
    print('')
    print('--------------------------------------------------------------')

In [10]:
def voc_ap(rec, prec, use_07_metric=True):
    """ ap = voc_ap(rec, prec, [use_07_metric])
    Compute VOC AP given precision and recall.
    If use_07_metric is true, uses the
    VOC 07 11 point method (default:True).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0]

        # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap


def voc_eval(detpath,
             annopath,
             imagesetfile,
             classname,
             cachedir,
             ovthresh=0.5,
             use_07_metric=True):
    """rec, prec, ap = voc_eval(detpath,
                           annopath,
                           imagesetfile,
                           classname,
                           [ovthresh],
                           [use_07_metric])
Top level function that does the PASCAL VOC evaluation.
detpath: Path to detections
   detpath.format(classname) should produce the detection results file.
annopath: Path to annotations
   annopath.format(imagename) should be the xml annotations file.
imagesetfile: Text file containing the list of images, one image per line.
classname: Category name (duh)
cachedir: Directory for caching the annotations
[ovthresh]: Overlap threshold (default = 0.5)
[use_07_metric]: Whether to use VOC07's 11 point AP computation
   (default True)
"""
# assumes detections are in detpath.format(classname)
# assumes annotations are in annopath.format(imagename)
# assumes imagesetfile is a text file with each line an image name
# cachedir caches the annotations in a pickle file
# first load gt
    if not os.path.isdir(cachedir):
        os.mkdir(cachedir)
    cachefile = os.path.join(cachedir, 'annots.pkl')
    # read list of images
    with open(imagesetfile, 'r') as f:
        lines = f.readlines()
    imagenames = [x.strip() for x in lines]
    if not os.path.isfile(cachefile):
        # load annots
        recs = {}
        for i, imagename in enumerate(imagenames):
            recs[imagename] = parse_rec(annopath % (imagename))
            if i % 100 == 0:
                print('Reading annotation for {:d}/{:d}'.format(
                   i + 1, len(imagenames)))
        # save
        print('Saving cached annotations to {:s}'.format(cachefile))
        with open(cachefile, 'wb') as f:
            pickle.dump(recs, f)
    else:
        # load
        with open(cachefile, 'rb') as f:
            recs = pickle.load(f)

    # extract gt objects for this class
    class_recs = {}
    npos = 0
    for imagename in imagenames:
        R = [obj for obj in recs[imagename] if obj['name'] == classname]
        bbox = np.array([x['bbox'] for x in R])
        difficult = np.array([x['difficult'] for x in R]).astype(np.bool)
        det = [False] * len(R)
        npos = npos + sum(~difficult)
        class_recs[imagename] = {'bbox': bbox,
                                 'difficult': difficult,
                                 'det': det}

    # read dets
    detfile = detpath.format(classname)
    with open(detfile, 'r') as f:
        lines = f.readlines()
    if any(lines) == 1:

        splitlines = [x.strip().split(' ') for x in lines]
        image_ids = [x[0] for x in splitlines]
        confidence = np.array([float(x[1]) for x in splitlines])
        BB = np.array([[float(z) for z in x[2:]] for x in splitlines])

        # sort by confidence
        sorted_ind = np.argsort(-confidence)
        sorted_scores = np.sort(-confidence)
        BB = BB[sorted_ind, :]
        image_ids = [image_ids[x] for x in sorted_ind]

        # go down dets and mark TPs and FPs
        nd = len(image_ids)
        tp = np.zeros(nd)
        fp = np.zeros(nd)
        for d in range(nd):
            R = class_recs[image_ids[d]]
            bb = BB[d, :].astype(float)
            ovmax = -np.inf
            BBGT = R['bbox'].astype(float)
            if BBGT.size > 0:
                # compute overlaps
                # intersection
                ixmin = np.maximum(BBGT[:, 0], bb[0])
                iymin = np.maximum(BBGT[:, 1], bb[1])
                ixmax = np.minimum(BBGT[:, 2], bb[2])
                iymax = np.minimum(BBGT[:, 3], bb[3])
                iw = np.maximum(ixmax - ixmin, 0.)
                ih = np.maximum(iymax - iymin, 0.)
                inters = iw * ih
                uni = ((bb[2] - bb[0]) * (bb[3] - bb[1]) +
                       (BBGT[:, 2] - BBGT[:, 0]) *
                       (BBGT[:, 3] - BBGT[:, 1]) - inters)
                overlaps = inters / uni
                ovmax = np.max(overlaps)
                jmax = np.argmax(overlaps)

            if ovmax > ovthresh:
                if not R['difficult'][jmax]:
                    if not R['det'][jmax]:
                        tp[d] = 1.
                        R['det'][jmax] = 1
                    else:
                        fp[d] = 1.
            else:
                fp[d] = 1.

        # compute precision recall
        fp = np.cumsum(fp)
        tp = np.cumsum(tp)
        rec = tp / float(npos)
        # avoid divide by zero in case the first detection matches a difficult
        # ground truth
        prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
        ap = voc_ap(rec, prec, use_07_metric)
    else:
        rec = -1.
        prec = -1.
        ap = -1.

    return rec, prec, ap

In [33]:
def getNoisyImage(orig):

    orig_shape = orig.shape
    noisy_orig = orig[...,::-1]/255.0
    noise =  np.random.normal(loc=0, scale=1, size=noisy_orig.shape)
    noise2 = (noise - noise.min())/(noise.max()-noise.min())

    # noise overlaid over image
    noisy = np.clip((noisy_orig + noise*0.2),0,1)
    noisy2 = np.clip((noisy_orig + noise*0.4),0,1)
    orig = noisy*255.0
    noisy_image = orig

    x = cv2.resize(orig, (300, 300)).astype(np.float32)
    x -= (104.0, 117.0, 123.0)
    x = x.astype(np.float32)
    x = x[:, :, ::-1].copy()
    x = torch.from_numpy(x).permute(2, 0, 1)

    return x, noisy_image

In [34]:
class NNRegressor(nt.NeuralNetwork):
    
    def __init__(self):
        super(NNRegressor, self).__init__()
        self.mse = nn.MSELoss()
        
    def criterion(self, y, d):
        return self.mse(y, d)

In [35]:
class DUDnCNN(NNRegressor):
    
    def __init__(self, D, C=64):
        super(DUDnCNN, self).__init__()
        self.D = D
        self.conv = nn.ModuleList()
        
        self.conv.append(nn.Conv2d(3, C, 3, padding=1)) # Padding = (3-1)/2 ==> 1
        
        for k in range(D):
            if (k == D/2):
                l = 1
            elif(k > D/2):
                l += 2
            else:
                l = 0
            
            dil = int(1 + 2**(k-l) -1)
            self.conv.append(nn.Conv2d(C, C, 3, padding=dil, dilation=dil))
        
        self.conv.append(nn.Conv2d(C, 3, 3, padding=1))
        
        self.bn = nn.ModuleList()
        for k in range(D):
            self.bn.append(nn.BatchNorm2d(C, C))
    
        for k in range(len(self.conv)-1):
            self.conv[k].weight.data = nn.init.kaiming_normal_(self.conv[k].weight.data, nonlinearity='relu')
        
        for k in range(D):    
            self.bn[k].weight.data = nn.init.constant_(self.bn[k].weight.data, 1.25 * np.sqrt(C))
        
    def forward(self, x):
        D = self.D
        h = F.relu(self.conv[0](x))
        for k in range(D):
            torch.backends.cudnn.benchmark=True  # Q20 
            h = self.conv[k+1](h)
            torch.backends.cudnn.benchmark=False  # Q20
            h = F.relu(self.bn[k](h))
        
        y = self.conv[D+1](h) + x
        return y

In [51]:
net_DUDnCNN = torch.load('../Assignment_4_FINAL/denNet.pt')

In [67]:
def getDenoised(noisy_image, net_DUDnCNN):
    denoised = noisy_image
    im = Image.fromarray(np.uint8(denoised))
    image_size = (300,300)

    # declare a tranform to convert the image to a tensor and normalize it between [-1, 1]
    transform = tv.transforms.Compose([tv.transforms.Resize(image_size),
                                tv.transforms.ToTensor(), tv.transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]) 
    
    denoised = transform(im).unsqueeze(0)
    
    if torch.cuda.is_available():
        denoised = denoised.cuda()
        net_DUDnCNN = net_DUDnCNN.cuda() 

    y = (net_DUDnCNN.forward(denoised)).detach()
    denoised = y[0].permute(1,2,0).cpu().numpy()
    denoised = 255.0*((denoised+1)/2)
    
    im2 = Image.fromarray(np.uint8(denoised))
    tran = tv.transforms.Compose([tv.transforms.Resize((noisy_image.shape[0],noisy_image.shape[1])), tv.transforms.ToTensor()])
    denoised = tran(im2)
    denoised = (tran(im2)*255.0).permute(1,2,0).numpy()

    x = cv2.resize(denoised, (300, 300)).astype(np.float32)
    x -= (104.0, 117.0, 123.0)
    x = x.astype(np.float32)
    x = x[:, :, ::-1].copy()
    x = torch.from_numpy(x).permute(2, 0, 1)
    
    return x, denoised

In [68]:
def test_net(save_folder, net, cuda, dataset, transform, top_k,
             im_size=300, thresh=0.05, noisy=False):
    
    num_images = len(dataset)
    # all detections are collected into:
    #    all_boxes[cls][image] = N x 5 array of detections in
    #    (x1, y1, x2, y2, score)
    all_boxes = [[[] for _ in range(num_images)]
                 for _ in range(len(labelmap)+1)]

    # timers
    _t = {'im_detect': Timer(), 'misc': Timer()}
    
    output_dir = "out/"
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    det_file = os.path.join(output_dir, 'detections_denoised.pkl')

    for i in range(num_images):

        if noisy:
            im, gt, h, w = dataset.pull_image(i)
            im, n = getNoisyImage(im)
            im, denoised_image = getDenoised(n, net_DUDnCNN)
        else:
            im, gt, h, w = dataset.pull_item(i)
            
        x = Variable(im.unsqueeze(0))
        if cuda:
            x = x.cuda()
        _t['im_detect'].tic()
        detections = net(x).data
        detect_time = _t['im_detect'].toc(average=False)

        # skip j = 0, because it's the background class
        for j in range(1, detections.size(1)):
            dets = detections[0, j, :]
            mask = dets[:, 0].gt(0.).expand(5, dets.size(0)).t()
            dets = torch.masked_select(dets, mask).view(-1, 5)
            if dets.size(0) == 0:
                continue
            boxes = dets[:, 1:]
            boxes[:, 0] *= w
            boxes[:, 2] *= w
            boxes[:, 1] *= h
            boxes[:, 3] *= h
            scores = dets[:, 0].cpu().numpy()
            cls_dets = np.hstack((boxes.cpu().numpy(),
                                  scores[:, np.newaxis])).astype(np.float32,
                                                                 copy=False)
            all_boxes[j][i] = cls_dets

        print('im_detect: {:d}/{:d} {:.3f}s'.format(i + 1,
                                                    num_images, detect_time))

    with open(det_file, 'wb') as f:
        pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)

    print('\nEvaluating detections')
    evaluate_detections(all_boxes, output_dir, dataset)
    
def evaluate_detections(box_list, output_dir, dataset):
    write_voc_results_file(box_list, dataset)
    do_python_eval(output_dir)

### EVALUATION

In [69]:
data_set = 'VOC'
voc_root = '//datasets/ee285f-public/PascalVOC2012/'
basenet = 'weights/vgg16_reducedfc.pth'
batch_size = 8
resume = None
start_iter = 0
num_workers = 4
cuda = True
learning_rate = lr = 1e-3
momentum = 0.9
weight_decay = 5e-4
gamma = 0.1
visdom = False
    
trained_model = 'ssd300_mAP_77.43_v2.pth'
save_folder = 'eval/'
if not os.path.exists(save_folder):
    os.mkdir(save_folder)
    
confidence_threshold = 0.01
top_k = 5
cuda = True
voc_root =  '//datasets/ee285f-public/PascalVOC2012/'
cleanup = True

In [70]:
annopath = os.path.join(voc_root, 'Annotations', '%s.xml')
imgpath = os.path.join(voc_root, 'JPEGImages', '%s.jpg')
imgsetpath = os.path.join(voc_root, 'ImageSets',
                          'Main', '{:s}.txt')
YEAR = '2012'
devkit_path = 'devkit_path/'
dataset_mean = (104, 117, 123)
set_type = 'val'

num_classes = len(labelmap) + 1                      # +1 for background
net = build_ssd('test', 300, num_classes)            # initialize SSD
net.load_state_dict(torch.load(trained_model))
net.eval()
print('Finished loading model!')
    # load data

dataset = VOCDetection(voc_root, [('2012', set_type)],
                BaseTransform(300, dataset_mean),VOCAnnotationTransform())

if cuda:
    net = net.cuda()
    cudnn.benchmark = True

# evaluation   
test_net(save_folder, net, cuda, dataset,
            BaseTransform(net.size, dataset_mean), top_k, 300,
             thresh=confidence_threshold, noisy=True)

Finished loading model!
im_detect: 1/5823 0.016s
im_detect: 2/5823 0.110s
im_detect: 3/5823 0.137s
im_detect: 4/5823 0.224s
im_detect: 5/5823 0.116s
im_detect: 6/5823 0.023s
im_detect: 7/5823 0.035s
im_detect: 8/5823 0.105s
im_detect: 9/5823 0.126s
im_detect: 10/5823 0.036s
im_detect: 11/5823 0.027s
im_detect: 12/5823 0.031s
im_detect: 13/5823 0.021s
im_detect: 14/5823 0.144s
im_detect: 15/5823 0.119s
im_detect: 16/5823 0.021s
im_detect: 17/5823 0.014s
im_detect: 18/5823 0.018s
im_detect: 19/5823 0.022s
im_detect: 20/5823 0.023s
im_detect: 21/5823 0.305s
im_detect: 22/5823 0.024s
im_detect: 23/5823 0.120s
im_detect: 24/5823 0.022s
im_detect: 25/5823 0.112s
im_detect: 26/5823 0.021s
im_detect: 27/5823 0.029s
im_detect: 28/5823 0.019s
im_detect: 29/5823 0.139s
im_detect: 30/5823 0.034s
im_detect: 31/5823 0.023s
im_detect: 32/5823 0.018s
im_detect: 33/5823 0.127s
im_detect: 34/5823 0.031s
im_detect: 35/5823 0.021s
im_detect: 36/5823 0.030s
im_detect: 37/5823 0.210s
im_detect: 38/5823 0.03

im_detect: 308/5823 0.015s
im_detect: 309/5823 0.109s
im_detect: 310/5823 0.018s
im_detect: 311/5823 0.099s
im_detect: 312/5823 0.021s
im_detect: 313/5823 0.109s
im_detect: 314/5823 0.024s
im_detect: 315/5823 0.142s
im_detect: 316/5823 0.029s
im_detect: 317/5823 0.216s
im_detect: 318/5823 0.129s
im_detect: 319/5823 0.023s
im_detect: 320/5823 0.123s
im_detect: 321/5823 0.027s
im_detect: 322/5823 0.039s
im_detect: 323/5823 0.123s
im_detect: 324/5823 0.023s
im_detect: 325/5823 0.097s
im_detect: 326/5823 0.120s
im_detect: 327/5823 0.022s
im_detect: 328/5823 0.018s
im_detect: 329/5823 0.024s
im_detect: 330/5823 0.018s
im_detect: 331/5823 0.235s
im_detect: 332/5823 0.099s
im_detect: 333/5823 0.138s
im_detect: 334/5823 0.019s
im_detect: 335/5823 0.014s
im_detect: 336/5823 0.117s
im_detect: 337/5823 0.025s
im_detect: 338/5823 0.044s
im_detect: 339/5823 0.119s
im_detect: 340/5823 0.037s
im_detect: 341/5823 0.032s
im_detect: 342/5823 0.027s
im_detect: 343/5823 0.027s
im_detect: 344/5823 0.019s
i

im_detect: 612/5823 0.094s
im_detect: 613/5823 0.047s
im_detect: 614/5823 0.020s
im_detect: 615/5823 0.121s
im_detect: 616/5823 0.101s
im_detect: 617/5823 0.025s
im_detect: 618/5823 0.130s
im_detect: 619/5823 0.254s
im_detect: 620/5823 0.143s
im_detect: 621/5823 0.028s
im_detect: 622/5823 0.131s
im_detect: 623/5823 0.138s
im_detect: 624/5823 0.121s
im_detect: 625/5823 0.113s
im_detect: 626/5823 0.022s
im_detect: 627/5823 0.108s
im_detect: 628/5823 0.039s
im_detect: 629/5823 0.018s
im_detect: 630/5823 0.018s
im_detect: 631/5823 0.018s
im_detect: 632/5823 0.131s
im_detect: 633/5823 0.029s
im_detect: 634/5823 0.040s
im_detect: 635/5823 0.205s
im_detect: 636/5823 0.024s
im_detect: 637/5823 0.119s
im_detect: 638/5823 0.026s
im_detect: 639/5823 0.230s
im_detect: 640/5823 0.237s
im_detect: 641/5823 0.122s
im_detect: 642/5823 0.021s
im_detect: 643/5823 0.024s
im_detect: 644/5823 0.029s
im_detect: 645/5823 0.032s
im_detect: 646/5823 0.016s
im_detect: 647/5823 0.045s
im_detect: 648/5823 0.127s
i

im_detect: 916/5823 0.153s
im_detect: 917/5823 0.029s
im_detect: 918/5823 0.111s
im_detect: 919/5823 0.020s
im_detect: 920/5823 0.025s
im_detect: 921/5823 0.029s
im_detect: 922/5823 0.112s
im_detect: 923/5823 0.026s
im_detect: 924/5823 0.208s
im_detect: 925/5823 0.039s
im_detect: 926/5823 0.097s
im_detect: 927/5823 0.131s
im_detect: 928/5823 0.027s
im_detect: 929/5823 0.023s
im_detect: 930/5823 0.041s
im_detect: 931/5823 0.035s
im_detect: 932/5823 0.046s
im_detect: 933/5823 0.116s
im_detect: 934/5823 0.024s
im_detect: 935/5823 0.280s
im_detect: 936/5823 0.019s
im_detect: 937/5823 0.053s
im_detect: 938/5823 0.023s
im_detect: 939/5823 0.094s
im_detect: 940/5823 0.029s
im_detect: 941/5823 0.126s
im_detect: 942/5823 0.037s
im_detect: 943/5823 0.118s
im_detect: 944/5823 0.028s
im_detect: 945/5823 0.122s
im_detect: 946/5823 0.017s
im_detect: 947/5823 0.121s
im_detect: 948/5823 0.019s
im_detect: 949/5823 0.115s
im_detect: 950/5823 0.017s
im_detect: 951/5823 0.032s
im_detect: 952/5823 0.020s
i

im_detect: 1212/5823 0.027s
im_detect: 1213/5823 0.124s
im_detect: 1214/5823 0.220s
im_detect: 1215/5823 0.018s
im_detect: 1216/5823 0.030s
im_detect: 1217/5823 0.133s
im_detect: 1218/5823 0.029s
im_detect: 1219/5823 0.121s
im_detect: 1220/5823 0.100s
im_detect: 1221/5823 0.017s
im_detect: 1222/5823 0.206s
im_detect: 1223/5823 0.016s
im_detect: 1224/5823 0.025s
im_detect: 1225/5823 0.125s
im_detect: 1226/5823 0.023s
im_detect: 1227/5823 0.018s
im_detect: 1228/5823 0.121s
im_detect: 1229/5823 0.133s
im_detect: 1230/5823 0.027s
im_detect: 1231/5823 0.033s
im_detect: 1232/5823 0.044s
im_detect: 1233/5823 0.019s
im_detect: 1234/5823 0.019s
im_detect: 1235/5823 0.039s
im_detect: 1236/5823 0.196s
im_detect: 1237/5823 0.021s
im_detect: 1238/5823 0.114s
im_detect: 1239/5823 0.207s
im_detect: 1240/5823 0.130s
im_detect: 1241/5823 0.121s
im_detect: 1242/5823 0.042s
im_detect: 1243/5823 0.050s
im_detect: 1244/5823 0.019s
im_detect: 1245/5823 0.016s
im_detect: 1246/5823 0.110s
im_detect: 1247/5823

...

im_detect: 5021/5823 0.018s
im_detect: 5022/5823 0.025s
im_detect: 5023/5823 0.115s
im_detect: 5024/5823 0.028s
im_detect: 5025/5823 0.028s
im_detect: 5026/5823 0.040s
im_detect: 5027/5823 0.114s
im_detect: 5028/5823 0.117s
im_detect: 5029/5823 0.183s
im_detect: 5030/5823 0.024s
im_detect: 5031/5823 0.019s
im_detect: 5032/5823 0.111s
im_detect: 5033/5823 0.112s
im_detect: 5034/5823 0.114s
im_detect: 5035/5823 0.125s
im_detect: 5036/5823 0.110s
im_detect: 5037/5823 0.034s
im_detect: 5038/5823 0.028s
im_detect: 5039/5823 0.022s
im_detect: 5040/5823 0.036s
im_detect: 5041/5823 0.018s
im_detect: 5042/5823 0.027s
im_detect: 5043/5823 0.215s
im_detect: 5044/5823 0.133s
im_detect: 5045/5823 0.118s
im_detect: 5046/5823 0.019s
im_detect: 5047/5823 0.117s
im_detect: 5048/5823 0.018s
im_detect: 5049/5823 0.030s
im_detect: 5050/5823 0.040s
im_detect: 5051/5823 0.041s
im_detect: 5052/5823 0.018s
im_detect: 5053/5823 0.114s
im_detect: 5054/5823 0.100s
im_detect: 5055/5823 0.295s
im_detect: 5056/5823

im_detect: 5314/5823 0.150s
im_detect: 5315/5823 0.223s
im_detect: 5316/5823 0.015s
im_detect: 5317/5823 0.044s
im_detect: 5318/5823 0.048s
im_detect: 5319/5823 0.174s
im_detect: 5320/5823 0.104s
im_detect: 5321/5823 0.022s
im_detect: 5322/5823 0.094s
im_detect: 5323/5823 0.047s
im_detect: 5324/5823 0.019s
im_detect: 5325/5823 0.108s
im_detect: 5326/5823 0.021s
im_detect: 5327/5823 0.138s
im_detect: 5328/5823 0.039s
im_detect: 5329/5823 0.048s
im_detect: 5330/5823 0.119s
im_detect: 5331/5823 0.028s
im_detect: 5332/5823 0.017s
im_detect: 5333/5823 0.023s
im_detect: 5334/5823 0.016s
im_detect: 5335/5823 0.038s
im_detect: 5336/5823 0.043s
im_detect: 5337/5823 0.033s
im_detect: 5338/5823 0.226s
im_detect: 5339/5823 0.019s
im_detect: 5340/5823 0.030s
im_detect: 5341/5823 0.122s
im_detect: 5342/5823 0.114s
im_detect: 5343/5823 0.021s
im_detect: 5344/5823 0.134s
im_detect: 5345/5823 0.125s
im_detect: 5346/5823 0.050s
im_detect: 5347/5823 0.028s
im_detect: 5348/5823 0.042s
im_detect: 5349/5823

im_detect: 5607/5823 0.016s
im_detect: 5608/5823 0.031s
im_detect: 5609/5823 0.129s
im_detect: 5610/5823 0.033s
im_detect: 5611/5823 0.024s
im_detect: 5612/5823 0.032s
im_detect: 5613/5823 0.028s
im_detect: 5614/5823 0.016s
im_detect: 5615/5823 0.019s
im_detect: 5616/5823 0.019s
im_detect: 5617/5823 0.023s
im_detect: 5618/5823 0.105s
im_detect: 5619/5823 0.021s
im_detect: 5620/5823 0.120s
im_detect: 5621/5823 0.138s
im_detect: 5622/5823 0.114s
im_detect: 5623/5823 0.024s
im_detect: 5624/5823 0.213s
im_detect: 5625/5823 0.031s
im_detect: 5626/5823 0.019s
im_detect: 5627/5823 0.146s
im_detect: 5628/5823 0.036s
im_detect: 5629/5823 0.032s
im_detect: 5630/5823 0.212s
im_detect: 5631/5823 0.099s
im_detect: 5632/5823 0.036s
im_detect: 5633/5823 0.025s
im_detect: 5634/5823 0.141s
im_detect: 5635/5823 0.123s
im_detect: 5636/5823 0.112s
im_detect: 5637/5823 0.030s
im_detect: 5638/5823 0.123s
im_detect: 5639/5823 0.243s
im_detect: 5640/5823 0.231s
im_detect: 5641/5823 0.246s
im_detect: 5642/5823

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


Writing boat VOC results file
Writing bottle VOC results file
Writing bus VOC results file
Writing car VOC results file
Writing cat VOC results file
Writing chair VOC results file
Writing cow VOC results file
Writing diningtable VOC results file
Writing dog VOC results file
Writing horse VOC results file
Writing motorbike VOC results file
Writing person VOC results file
Writing pottedplant VOC results file
Writing sheep VOC results file
Writing sofa VOC results file
Writing train VOC results file
Writing tvmonitor VOC results file
VOC07 metric? Yes
AP for aeroplane = 0.8283
AP for bicycle = 0.8459
AP for bird = 0.7313
AP for boat = 0.5694
AP for bottle = 0.5789
AP for bus = 0.8393
AP for car = 0.7014
AP for cat = 0.8711
AP for chair = 0.5764
AP for cow = 0.7177
AP for diningtable = 0.7320
AP for dog = 0.8331
AP for horse = 0.8186
AP for motorbike = 0.8715
AP for person = 0.7467
AP for pottedplant = 0.4905
AP for sheep = 0.6729
AP for sofa = 0.7417
AP for train = 0.8270
AP for tvmonitor